# CITS4404 Practical Project

### Camo Worms!

<div style="margin-left: 0px">
    &nbsp;
    <img src="CamoWorms/velvet-worms.jpg" width=600>
    <br>
    <br>
</div>

### Imports and Global Variables

In [ ]:
# Imports
# pip install imageio matplotlib scikit-learn scipy sympy
import random
import numpy as np
import imageio.v3 as iio

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.bezier as mbezier

from sklearn.metrics.pairwise import euclidean_distances

rng = np.random.default_rng()
Path = mpath.Path
mpl.rcParams['figure.dpi']= 72 #size of images

In [ ]:
# Global Variables
IMAGE_DIR = 'MendeleyDataset'
IMAGE_NAME='DME-1102486-2'
MASK = [0, 1000, 0, 1000] # ymin ymax xmin xmax
IMAGE_DIR = 'CamoWorms/images'
IMAGE_NAME='original'
MASK = [320, 560, 160, 880] # ymin ymax xmin xmax

### Image Preperation

In [ ]:
# Read, crop and display image and stats

def crop (image, mask):
    h, w = np.shape(image)
    return image[max(mask[0],0):min(mask[1],h), max(mask[2],0):min(mask[3],w)]

def prep_image (imdir, imname, mask):
    print("Image name (shape) (intensity max, min, mean, std)\n")
    image = np.flipud(crop(iio.imread(imdir+'/'+imname+".png"), mask))
    print("{} {} ({}, {}, {}, {})".format(imname, np.shape(image), np.max(image), np.min(image), round(np.mean(image),1), round(np.std(image),1)))
    plt.imshow(image, vmin=0, vmax=255, cmap='gray', origin='lower') # use vmin and vmax to stop imshow from scaling
    plt.show()
    return image

### Camo Worms and Drawing Them

In [ ]:
class Camo_Worm:
    def __init__(self, x, y, r, theta, deviation_r, deviation_gamma, width, colour):
        self.x = x
        self.y = y
        self.r = r
        self.theta = theta
        self.dr = deviation_r
        self.dgamma = deviation_gamma
        self.width = width
        self.colour = colour
        p0 = [self.x - self.r * np.cos(self.theta), self.y - self.r * np.sin(self.theta)]
        p2 = [self.x + self.r * np.cos(self.theta), self.y + self.r * np.sin(self.theta)]
        p1 = [self.x + self.dr * np.cos(self.theta+self.dgamma), self.y + self.dr * np.sin(self.theta+self.dgamma)]
        self.bezier = mbezier.BezierSegment(np.array([p0, p1,p2]))

    def control_points (self):
        return self.bezier.control_points

    def path (self):
        return mpath.Path(self.control_points(), [Path.MOVETO, Path.CURVE3, Path.CURVE3])

    def patch (self):
        return mpatches.PathPatch(self.path(), fc='None', ec=str(self.colour), lw=self.width, capstyle='round')

    def intermediate_points (self, intervals=None):
        if intervals is None:
            intervals = max(3, int(np.ceil(self.r/8)))
        return self.bezier.point_at_t(np.linspace(0,1,intervals))

    def approx_length (self):
        intermediates = self.intermediate_points()
        eds = euclidean_distances(intermediates,intermediates)
        return np.sum(np.diag(eds,1))

    def colour_at_t(self, t, image):
        intermediates = np.int64(np.round(np.array(self.bezier.point_at_t(t)).reshape(-1,2)))
        colours = [image[point[1],point[0]] for point in intermediates]
        return(np.array(colours)/255)


In [ ]:
class Drawing:
    def __init__ (self, image):
        self.fig, self.ax = plt.subplots()
        self.image = image
        self.im = self.ax.imshow(self.image, cmap='gray', origin='lower')

    def add_patches(self, patches):
        try:
            for patch in patches:
                self.ax.add_patch(patch)
        except TypeError:
            self.ax.add_patch(patches)

    def add_dots(self, points, radius=4, **kwargs):
        try:
            for point in points:
                self.ax.add_patch(mpatches.Circle((point[0],point[1]), radius, **kwargs))
        except TypeError:
            self.ax.add_patch(mpatches.Circle((points[0],points[1]), radius, **kwargs))

    def add_worms(self, worms):
        try:
            self.add_patches([w.patch() for w in worms])
        except TypeError:
            self.add_patches([worms.patch()])

    def show(self, save=None):
        if save is not None:
            plt.savefig(save)
        plt.show()

### Random Worms and Clews

In [ ]:
# Example of a random worm. You may do this differently.

    # centre points, angles and colour chosen from uniform distributions
    # lengths chosen from normal distributions with two std parameters passed
    # width chosen from gamma distribution with shape parameter 3 and scale passed

def random_worm (imshape, init_params):
    (radius_std, deviation_std, width_theta) = init_params
    (ylim, xlim) = imshape
    midx = xlim * rng.random()
    midy = ylim * rng.random()
    r = radius_std * np.abs(rng.standard_normal())
    theta = rng.random() * np.pi
    dr = deviation_std * np.abs(rng.standard_normal())
    dgamma = rng.random() * np.pi
    colour = rng.random()
    width = width_theta * rng.standard_gamma(3)
    return Camo_Worm(midx, midy, r, theta, dr, dgamma, width, colour)

In [ ]:
# Initialise a random clew

def initialise_clew (size, imshape, init_params):
    clew = []
    for i in range(size):
        clew.append(random_worm(imshape, init_params))
    return clew

&copy; Cara MacNish, UWA, 2024

## Cost Function

In [ ]:
# class CostFunction:
#     def __init__(self, image, weights):
#         self.image = image
#         self.weights = weights

#     def coverage_cost(self, worm):
#         pass

#     def smoothness_cost(self, worm):
#         '''
#         Considering the worm's curvature and length with respect to 
#         the layer changes (i.e., intensity changes).
#         '''
#         total_smoothness_cost = 0
#         detail = 100
#         ideal_length = 300
#         # ideal_curves = {"no-edge": 200, "slight-edge": 100, "edge": 10}
#         path_intensity = [[],[]]

#         # Get 100 points along the worm's path
#         path = np.array(worm.intermediate_points(detail))
#         p0 ,p1 = worm.control_points()[0], worm.control_points()[2]
#         mid_point = [(p0[0]+p1[0])/2, (p0[1]+p1[1])/2]

#         # Calculate color adaptation cost for each point
#         for x, y in path:
#             # Ensure coordinates are within the image bounds
#             x = max(0, min(int(x), self.image.shape[1] - 1))
#             y = max(0, min(int(y), self.image.shape[0] - 1))
#             # Sort intensity at each point
#             pixel_colour = self.image[y, x].astype(float) / 255
#             path_intensity[1].append([x,y])
#             path_intensity[0].append(pixel_colour)

#         # Calculate the intesity gradients and sum
#         gradients = abs(np.diff(path_intensity[0]))
#         edge_point = path_intensity[1][np.argmax(gradients)]
#         gradient_sum = sum(gradients)
#         gradient_avg = np.average(gradients)
        

#         # Get the worm's features
#         worm_curvature = worm.dr
#         worm_curvature_point = worm.control_points()[1]
#         worm_curve_direct = worm.dgamma
#         worm_orientation = worm.theta
#         worm_length = worm.approx_length()

#         print("################# NEW WORM #################")
#         print(f"path_intensity:{path_intensity}")
#         print(f"gradient_avg:{gradient_avg}")
#         print(f"gradient:{gradients}")
#         print(f"gradient_sum:{gradient_sum}")
#         drawing = Drawing(self.image)
#         drawing.add_worms(worm)
#         drawing.add_dots(worm.control_points(),color='orange')
#         drawing.add_dots(edge_point,color='blue')
#         drawing.add_dots(path_intensity[1][edge_points[0][0]],color='red')
#         drawing.show()
        
#         total_smoothness_cost += abs(ideal_length-worm_length)/ideal_length
#         print(f"MAGNITUDE:{gradient_sum}\nSMOOTHNESS COST: {total_smoothness_cost}\n")
#         print("############################################")
#         return total_smoothness_cost


#     def detail_preservation_cost(self, worm):
#         pass

#     def colour_adaptation_cost(self, worm):
#         # Initialize total colour cost
#         total_colour_cost = 0

#         # Get all points along the worm's path
#         num_points = max(int(worm.approx_length() / (worm.width)), 1)
#         path = worm.intermediate_points(num_points)

#         # Get worm's colour and width
#         colour = worm.colour
#         width = int(worm.width)
#         checked_points = []
#         total_colour_cost_added = 0

#         # Calculate colour adaptation cost for each point
#         for x, y in path:
#             # Ensure coordinates are within the image bounds
#             x = max(0, min(int(x), self.image.shape[1] - 1))
#             y = max(0, min(int(y), self.image.shape[0] - 1))

#             for dx in range(-width, width + 1):
#                 for dy in range(-width, width + 1):
#                     if (dx, dy) not in checked_points:
#                         checked_points.append((dx, dy))
#                         # Check coordinates are within the image bounds
#                         nx = max(0, min(int(x + dx), self.image.shape[1] - 1))
#                         ny = max(0, min(int(y + dy), self.image.shape[0] - 1))

#                         # Calculate the distance between current point and the worm's closest intermediate point
#                         distance = np.sqrt((x - nx) ** 2 + (y - ny) ** 2)

#                         # If the distance is less than the worm's width, calculate the color difference
#                         if distance <= width:
#                             # Calculate colour difference between worm and image at each point
#                             image_colour = self.image[ny, nx].astype(float) / 255
#                             colour_difference = np.linalg.norm(colour - image_colour)
                            
#                             # Accumulate colour adaptation cost
#                             total_colour_cost += colour_difference
#                             total_colour_cost_added += 1

#         # ------------------Testing ---------------------
#         # Change the return value to the value below -- just for testing purposes                    
#         return (1 - colour) + total_colour_cost / total_colour_cost_added
#         # return total_colour_cost / total_colour_cost_added

#     def efficiency_cost(self, worm):
#         pass

#     def evaluate(self, worm):
#         # Evaluate the total cost
#         coverage_cost = 0 #self.coverage_cost(worm)
#         smoothness_cost = self.smoothness_cost(worm)
#         detail_cost = 0 #self.detail_preservation_cost(worm)
#         colour_cost = self.colour_adaptation_cost(worm)
#         efficiency_cost = 0 #self.efficiency_cost(worm)
#         total_cost = (self.weights['coverage'] * coverage_cost +
#                       self.weights['smoothness'] * smoothness_cost +
#                       self.weights['detail'] * detail_cost +
#                       self.weights['colour'] * colour_cost +
#                       self.weights['efficiency'] * efficiency_cost)

#         return total_cost

In [ ]:
class CostFunction:
    '''
    Calculates the cost of each worm, and returns the 
    '''
    def __init__(self, image, weights, info=True):
        self.image = image
        self.weights = weights
        # self.ideal_length = 0.2 * image.shape[1]

    def internal_cost(self, worm):
        '''
        Considers standardised ideals for the worm's properties.
        '''
        # ideal_length = 100
        # total_length_cost = 3 * abs(ideal_length - worm.approx_length())/ideal_length
        
        if abs(worm.dr) > 1.5*worm.r:
            ideal_curvature = 0.5 * worm.r 
            total_curvature_cost = 0.05 * abs(abs(ideal_curvature) - worm.dr)/ideal_curvature
        else:
            total_curvature_cost = 0
        # ideal_curvature = 0.5 * worm.r 
        # total_curvature_cost = 0.05 * abs(abs(ideal_curvature) - worm.dr)/ideal_curvature
        # ideal_width = 7
        # total_width_cost = abs(ideal_width - worm.width)/ideal_width
        
        # total_curvature_cost = worm.dr / worm.r
        total_curve_angle_cost = abs(worm.dgamma) / (2*np.pi)
        total_length_cost = 100 / worm.approx_length()
        total_width_cost = 1 / (0.5 * worm.width)

        print(f"------------------------------------------INTERNAL--")
        print(f"Curvature_cost: {total_curvature_cost}")
        print(f"Curvature_Angle_cost: {total_curve_angle_cost}")
        print(f"Length_cost: {total_length_cost}")
        print(f"Width_cost: {total_width_cost}")
        return total_curvature_cost + total_length_cost + total_width_cost + total_curve_angle_cost
    
    def group_cost(self, worm, clew):
        '''
        Considers the effect of clew proximity on the worm.
        '''
        total_overlap_cost = 0

        p0 ,p1 = worm.control_points()[0], worm.control_points()[2]
        worm_midpoint = [(p0[0]+p1[0])/2, (p0[1]+p1[1])/2]
        worm_midpoint = worm.control_points()[1]

        clew_midpoints = np.array([w.control_points()[1] for w in clew])

        space = 10
        x_space_invasion = np.where(((worm_midpoint[0] - space) < clew_midpoints[:, 0]) & (clew_midpoints[:, 0] < (worm_midpoint[0] + space)))
        y_space_invasion = np.where(((worm_midpoint[1] - space) < clew_midpoints[:, 1]) & (clew_midpoints[:, 1] < (worm_midpoint[1] + space)))
        
        total_overlap_cost += (len(x_space_invasion[0]) + len(y_space_invasion[0]))
        
        print(f"---------------------------------------------GROUP--")
        print(f"Overlap_cost: {total_overlap_cost}")
        return total_overlap_cost
    
    def environment_cost(self, worm):
        '''
        Considers the effect of the image on the worm's properties.
        '''
        print(f"-----------------------------------------------ENV--")
        # Initialise total costs
        total_colour_cost = 0
        total_out_of_bounds_cost = 0
        total_edge_cost = 0
        total_location_cost = 0
        total_width_cost = 0

        # Get all points along the worm's path
        num_points = max(int(worm.approx_length() / (worm.width)), 1)
        path = worm.intermediate_points(num_points)

        # Get worm's colour and width
        colour = worm.colour
        width = int(worm.width)
        checked_points = []
        total_colour_cost_added = 0

        # Calculate colour adaptation cost for each point
        for x, y in path:
            # Ensure coordinates are within the image bounds
            x = max(0, min(int(x), self.image.shape[1] - 1))
            y = max(0, min(int(y), self.image.shape[0] - 1))

            for dx in range(-width, width + 1):
                for dy in range(-width, width + 1):
                    if (dx, dy) not in checked_points:
                        checked_points.append((dx, dy))
                        # Check coordinates are within the image bounds
                        nx = max(0, min(int(x + dx), self.image.shape[1] - 1))
                        ny = max(0, min(int(y + dy), self.image.shape[0] - 1))

                        # Calculate the distance between current point and the worm's closest intermediate point
                        distance = np.sqrt((x - nx) ** 2 + (y - ny) ** 2)

                        # If the distance is less than the worm's width, calculate the color difference
                        if distance <= width:
                            # Calculate colour difference between worm and image at each point
                            image_colour = self.image[ny, nx].astype(float) / 255
                            colour_difference = np.linalg.norm(colour - image_colour)
            
                            # Accumulate colour adaptation cost
                            total_colour_cost += colour_difference
                            total_colour_cost_added += 1

        # # ------------------Testing ---------------------
        # # Change the return value to the value below -- just for testing purposes                    
        # return (1 - colour) + total_colour_cost / total_colour_cost_added
        total_colour_cost = 3 * (total_colour_cost / total_colour_cost_added)


        # Get points along the worm's path
        detail = 100
        path_colours = [[],[]]
        path = np.array(worm.intermediate_points(detail))

        # Calculate color adaptation cost for each point
        for x, y in path:
            # Ensure coordinates are within the image bounds
            outside = False
            if (int(x) > (self.image.shape[1] - 1)) or (int(x) < 0):
                total_out_of_bounds_cost += 0.07
                outside = True
            if (int(y) > (self.image.shape[0] - 1)) or (int(y) < 0):
                total_out_of_bounds_cost += 0.07
                outside = True
            if outside:
                continue
            x = max(0, min(int(x), self.image.shape[1] - 1))
            y = max(0, min(int(y), self.image.shape[0] - 1))
            # Sort intensity at each point
            pixel_colour = self.image[y, x].astype(float) / 255
            path_colours[1].append([x,y])
            path_colours[0].append(pixel_colour)

        # Calculate the intesity gradients and related cost
        gradients = abs(np.diff(path_colours[0]))
        if len(gradients) > 0:
            gradient_avg = np.mean(gradients)
            total_edge_cost += gradient_avg * 50
            edge_points = len(np.where(gradients >= 0.08)[0])
            print(f"Edge_gradient: {max(gradients)}")
            if edge_points > 0.10 * detail :
                total_edge_cost += 2

        # # Calculate worm location cost 
        # median_colour = np.mean(path_colours[0])
        # if median_colour < 0.15:
        #     total_location_cost += 3
                
        # Calculate worm colour deviation cost 
        colour_deviation = np.std(path_colours[0])
        if colour_deviation:
            if colour_deviation > 0.1:
                total_location_cost += 10 * colour_deviation
            else:
                total_width_cost += 1 / (0.5 * worm.width)
        
        print(f"Colour_deviation: {colour_deviation}")
        print(f"Colour_cost: {total_colour_cost}")
        print(f"Edge_cost: {total_edge_cost}")
        print(f"OutOfBounds_cost: {total_out_of_bounds_cost}")
        print(f"Width_cost: {total_width_cost}")
        # print(f"Location_cost: {total_location_cost}")
        return total_colour_cost + total_edge_cost + total_out_of_bounds_cost + total_location_cost + total_width_cost
    
    def evaluate (self, clew, info=True):
        total_cost = []
        count = 0
        for worm in clew:
            count += 1

            # Testing: Display Worm information
            if info:
                print(f"====================================================")
                print(f"Worm No.{count}")
                print(f"({worm.x}, {worm.y})")
                # drawing = Drawing(self.image)
                # drawing.add_worms(worm)
                # drawing.add_dots(worm.control_points(),color='orange')
                # drawing.show()

            internal = self.weights["internal"] * self.internal_cost(worm)
            group = self.weights["group"] * self.group_cost(worm, clew)
            environment = self.weights["environment"] * self.environment_cost(worm)
            total_cost.append(internal + group + environment)

            if info:
                print(f"----------------------------------------------------")
                print(f"[i]\tTotal Internal: {internal}")
                print(f"[g]\tTotal Group: {group}")
                print(f"[e]\tTotal Environment: {environment}")
                print(f"[!]\tWORM COST: {internal + group + environment}")
                print(f"====================================================\n")

        if info:
            drawing = Drawing(self.image)
            drawing.add_worms(clew)

            temp = np.array(total_cost)
            index = np.argmin(temp)
            drawing.add_worms(clew[index])
            drawing.add_dots(clew[index].control_points(),color='blue')

            index = np.argmax(temp)
            drawing.add_worms(clew[index])
            drawing.add_dots(clew[index].control_points(),color='red')

            drawing.show()
        return total_cost
            


### Genetic Algorithm

In [ ]:
class GeneticAlgorithm:
    def __init__(self, image, population_size, crossover_rate, mutation_rate, weights):
        self.image = image
        self.population_size = population_size 
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.weights = weights

    def initialise_population(self, image_shape, init_params):
        population = [random_worm(image_shape, init_params) for _ in range(self.population_size)]
        return population

    def select_parents(self, population):
        # Select parents based on fitness
        tournament_size = 3
        parents = []
        for _ in range(self.population_size):
            # ------------ Replace with Different Function------------
            tournament_indices = np.random.choice(range(len(population)), tournament_size, replace=False)
            tournament = [population[i] for i in tournament_indices]
            winner = random.choice(tournament)
            parents.append(winner)
        return parents

    def crossover(self, parent1, parent2):
        # Crossover parents to create offspring
        params = ['x', 'y', 'r', 'theta', 'dr', 'dgamma', 'width', 'colour']

        # Get other parameters from parents
        p1 = np.zeros(len(params))
        p2 = np.zeros(len(params))
        for i, param in enumerate(params):
            if random.random() < self.crossover_rate:
                p1[i] = getattr(parent2, param)
                p2[i] = getattr(parent1, param)
            else:
                p1[i] = getattr(parent1, param)
                p2[i] = getattr(parent2, param)
        
        return p1, p2

    def mutate(self, worm_params, init_params):
        # Mutate offspring
        # Randomly select any parameters of the worm and mutate it
        params = ['x', 'y', 'r', 'theta', 'dr', 'dgamma', 'width', 'colour']
        (radius_std, deviation_std, width_theta) = init_params
        for i, param in enumerate(params):
            if random.random() < self.mutation_rate:
                if param == 'x':
                    worm_params[i] = self.image.shape[1] * rng.random()
                elif param == 'y':
                    worm_params[i] = self.image.shape[0] * rng.random()
                elif param == 'r':
                    worm_params[i] = radius_std * np.abs(rng.standard_normal())
                elif param == 'theta':
                    worm_params[i] = rng.random() * np.pi
                elif param == 'dr':
                    worm_params[i] = deviation_std * np.abs(rng.standard_normal())
                elif param == 'dgamma':
                    worm_params[i] = rng.random() * np.pi
                elif param == 'width':
                    worm_params[i] = width_theta * rng.standard_gamma(3)
                elif param == 'colour':
                    worm_params[i] = rng.random()
        return worm_params
    
    def run(self, max_generations, init_params):
        # Initialize population
        population = self.initialise_population(self.image.shape, init_params)
        drawing = Drawing(self.image)
        drawing.add_worms(population)
        drawing.show()

        cost = CostFunction(self.image, self.weights)

         # ------------ Add Cost_function to offspring------------
        for _ in range(max_generations):
            # Select parents
            parents = self.select_parents(population)

            # Create offspring through crossover
            offspring = []
            for i in range(0, self.population_size, 2):
                child_param1, child_param2 = self.crossover(parents[i], parents[i+1])
                child_param1= self.mutate(child_param1, init_params)
                child_param2= self.mutate(child_param2, init_params)
                child1 = Camo_Worm(*child_param1)
                child2 = Camo_Worm(*child_param2)
                offspring.extend([child1, child2])
            
            # add parents and offspring together
            new_population = population + offspring

            # Evaluate the cost of each worm in the new population
            # pop_cost = [cost.evaluate(worm) for worm in new_population]
            pop_cost = cost.evaluate(new_population)

            # Get the indices of worms sorted by their costs in ascending order
            sorted_indices = sorted(range(len(pop_cost)), key=lambda k: pop_cost[k])
            sorted_population = [new_population[i] for i in sorted_indices]

            # Replace old population with new population
            population = sorted_population[:self.population_size]

        return population

### Run Genetic Algorithm

In [ ]:
# Load in OCT scan
image = prep_image(IMAGE_DIR, IMAGE_NAME, MASK)

# Define parameters for the Genetic Algorithm
population_size = 30 # Number of worms in the population ---- to be adjusted
crossover_rate = 0.5 # Probability of crossover
mutation_rate = 0.1 # Probability of mutation`

# Define weights --- to be adjusted/required?
# weights = {'coverage': 1, 'smoothness': 1, 'detail': 1, 'colour': 1, 'efficiency': 1}
weights = {'internal': 0.65, 'group': 1, 'environment': 1}
# Create an instance of the GeneticAlgorithm class
genetic_algorithm = GeneticAlgorithm(image, population_size, crossover_rate, mutation_rate, weights)

# Define initial parameters for the worms
init_params = (40, 30, 1)  # Example parameters, adjust as needed

# Define the maximum number of generations
max_generations = 60
clew = genetic_algorithm.run(max_generations, init_params)

# Display the optimized population of worms
drawing = Drawing(image)
for worm in clew:
    drawing.add_worms(worm)
    drawing.add_dots(worm.control_points(), color="red")


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Testing cost function for a single worm
worm_test = Camo_Worm(100, 180, 50, 0, 50, np.pi/2, 10, 0.9)
print(np.pi/2, np.pi/9)
worm_test2 = Camo_Worm(200, 160, 50, np.pi/6, 70, np.pi/1, 8, 0.6)
test = ['x', 'y', 'r', 'theta', 'dr', 'dgamma', 'width', 'colour']
# print(worm_test)
# cost = CostFunction(image, weights)
# c_c = cost.evaluate(worm_test)
# print(c_c)
# drawing.add_worms(worm_test)

# drawing.add_dots(worm_test.intermediate_points(300), radius=2, color='green')
# drawing.add_dots(worm_test.control_points(),color='orange')
print(f'length:{worm_test.approx_length()}')
print(f"control points:\n{worm_test.control_points()}")

drawing.show()